In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1=nn.Conv2d(1,10,kernel_size=5)
        self.conv2=nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop=nn.Dropout2d()
        self.fc1=nn.Linear(320,50)
        self.fc2=nn.Linear(50,10)
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1(x),2))
        x=F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x=x.view(-1,320)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)

In [3]:
model=LeNet()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.005,momentum=0.9)

In [4]:
cuda_gpu=torch.cuda.is_available()

In [5]:
torch.cuda.is_available()

False

In [6]:
def train(model,epoch,criterion,optimizer,data_loader):
    model.train()
    for batch_idx,(data,target) in enumerate(data_loader):
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        optimizer.zero_grad()
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()

In [7]:
from torchvision import datasets,transforms
batch_num_size=64
train_loader=torch.utils.data.DataLoader(datasets.MNIST('data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),batch_size=batch_num_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(datasets.MNIST('data',train=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),batch_size=batch_num_size,shuffle=True)

In [8]:
def test(model,epoch,criterion,data_loader):
    model.eval()
    test_loss=0
    correct=0
    for data,target in data_loader:
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        test_loss+=criterion(output,target).item()
        pred=output.data.max(1)[1]
        correct+=pred.eq(target.data).cpu().sum()
    test_loss/=len(data_loader)
    acc=correct/len(data_loader.dataset)
    return (acc,test_loss),output.data.max(1)[1]

In [16]:
from PIL import Image
import numpy as np

In [17]:
im=Image.open('C:\\Users\\zhou\\Desktop\\play3.jpg')

In [18]:
r,g,b=im.split()

In [19]:
r=np.array(r)

In [20]:
#PCA数据降维
def PCA(matX,k):
    average=np.mean(matX,axis=0)
    m,n=matX.shape
    avgs=np.tile(average,(m,1))
    data_adjust=matX-avgs
    data_adjust=np.array(data_adjust,dtype=float)
    covX=np.cov(data_adjust.T)
    feaValue,feaVec=np.linalg.eig(covX)
    index=np.argsort(-feaValue)
    selectVec=np.matrix(feaVec.T[index[:k]])
    final=matX*selectVec.T
    return(final)

In [21]:
model.eval()
def input_pic(image):
    r,g,b=image.split()
    r=np.array(r)
    r1=PCA(r,28)
    r=r1.T
    r=PCA(r,28).T
    r=(r-r.min(0))/(r.max(0)-r.min(0))
    r=torch.tensor(r)
    r=Variable(r).float()
    r=r.reshape(1,1,28,28)
    final=model(r)
    return final.data.max(1)[1]

In [ ]:
input_pic(im)

In [9]:
epochs=5

In [10]:
train(model,100,criterion,optimizer,train_loader)

In [11]:
test(model,epochs,criterion,test_loader) 

((tensor(0.9608), 0.11767887171049407),
 tensor([9, 1, 6, 5, 2, 3, 8, 9, 0, 2, 4, 4, 7, 6, 6, 7]))

In [ ]:
if os.path.isfile('C:\\Users\\zhou\\Desktop\\nn.pth'):
    print('loading model')
    model.load_state_dict(torch.load('C:\\Users\\zhou\\Desktop\\nn.pth',map_location=lambda storage,loc:storage))
    acc,loss=test(model,1,criterion,test_loader)
else:
    print('Training model')
    for epoch in range(1,epochs+1):
        train(model,epoch,criterion,optimizer,train_loader)
        acc,loss =test(model,1,criterion,test_loader)
        torch.save(model.state_dict(),'C:\\Users\\zhou\\Desktop\\nn.pth')
data=model.conv1.weight.cpu().data.numpy()


In [ ]:
kernel_num=data.shape[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
checkpoint=torch.load('C:\\Users\\zhou\\Desktop\\nn.pth')

In [ ]:
list(checkpoint)

In [ ]:
list1=[]
list2=[]
for i,j in train_loader:
    list1.append(i)
    list2.append(j)